# Token Estimator

A notebook to aid with the cost estimations for this benchmarks. We estimate the number of tokens based on GPT-4o's tokenizer and then compute the API costs for various models based on current rates. 

In [2]:
# we run the notebook from the top-level of the repo
%cd ..

/home/sherbold/git/sortbench


/home/sherbold/git/sortbench/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
import tiktoken

from sortbench.util.data_utils import load_data_local

enc = tiktoken.encoding_for_model("gpt-4o")

modes = ["basic", "advanced", "debug"]
tokens_per_mode = {}

for mode in modes:
    configs = load_data_local(file_path="benchmark_data", name="sortbench", mode=mode, version="v1.0")
    for config in configs.values():
        for list in config.values():
            if mode not in tokens_per_mode:
                tokens_per_mode[mode] = 0
            tokens_per_mode[mode] += len(enc.encode(f"{list}"))
    print(f"Total tokens for {mode} mode: {tokens_per_mode[mode]}")

total_tokens = sum(tokens_per_mode.values())
print(f"Total tokens: {total_tokens}")
print()

model_costs = {"gpt-4o": {"input": 2.0, "output": 10.0},
               "gpt-o1": {"input": 15.0, "output": 60.0},
               "gpt-4o-mini": {"input": 0.15, "output": 0.6},
               "gpt-3.5-turbo": {"input": 3.0, "output": 6.0}
               }

total_costs = 0
for model, costs in model_costs.items():
    costs_input = (total_tokens * costs['input']) / 1000000
    costs_output = (total_tokens * costs['output']) / 1000000
    costs_model = costs_input + costs_output    
    total_costs += costs_model

    print(f"Model: {model}")
    print(f"Input costs: {costs_input}")
    print(f"Output costs: {costs_output}")
    print(f"Total costs: {costs_model}")
    print("")
print(f"Total costs: {total_costs}")

Total tokens for basic mode: 88430
Total tokens for advanced mode: 319621
Total tokens for debug mode: 176512
Total tokens: 584563

Model: gpt-4o
Input costs: 1.169126
Output costs: 5.84563
Total costs: 7.014756

Model: gpt-o1
Input costs: 8.768445
Output costs: 35.07378
Total costs: 43.842225

Model: gpt-4o-mini
Input costs: 0.08768445
Output costs: 0.3507378
Total costs: 0.43842225

Model: gpt-3.5-turbo
Input costs: 1.753689
Output costs: 3.507378
Total costs: 5.261067000000001

Total costs: 56.556470250000004
